In [2]:
#
# NOTE: This machine learning program is for predicting TC formation, using
#       input dataset in the NETCDF format. The program treats different
#       2D input fields as different channels of an image. This specific
#       program requires a set of 12 2D-variables (12-channel image) and
#       consists of three stages
#       - Stage 1: reading NETCDF input and generating (X,y) data with a
#                  given image sizes, which are then saved by pickle;
#       - Stage 2: import the saved pickle (X,y) pair and build a CNN model
#                  with a given training/validation ratio, and then save
#                  the train model under tcg_CNN.model.
#       - Stage 3: import the trained model from Stage 2, and make a list
#                  of prediction from normalized test data.
#
# INPUT: This Stage 3 script reads in the CNN trained model "tcg_CNN.model"
#        that is generated from Step 2.
#
#        Remarks: Note that the input data for this script must be on the
#        same as in Step 1 with standard 19 vertical
#        levels 1000, 975, 950, 925, 900, 850, 800, 750, 700, 650, 600,
#        550, 500, 450, 400, 350, 300, 250, 200. Also, all field vars must
#        be resize to cover an area of 30x30 around the TC center for the
#        positive data cases.
#        Similar to Step 2, this Step 3 needs to also have a large mem
#        allocation so that it can be run properly.
#
# OUTPUT: A list of probability forecast with the same dimension as the
#        number of input 12-channel images.
#
# HIST: - 01, Nov 22: Created by CK
#       - 02, Nov 22: Modified to optimize it
#       - 05. Jun 23: Rechecked and added F1 score function for a list of models
#
# AUTH: Chanh Kieu (Indiana University, Bloomington. Email: ckieu@iu.edu)
#
#==========================================================================
import cv2
import tensorflow as tf
import os
from tqdm import tqdm
import netCDF4
import numpy as np

In [3]:
#
# Function to return input data as an numpy array
#
def prepare(filepath):
    IMG_SIZE = 30
    number_channels = 12
    f = netCDF4.Dataset(filepath)
    abv = f.variables['absvprs']
    nx = np.size(abv[0,0,:])
    ny = np.size(abv[0,:,0])
    nz = np.size(abv[:,0,0])
    #print('Dimension of input NETCDF is: ',nx,ny,nz)
    a2 = np.zeros((nx,ny,number_channels))
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,0] = abv[1,j,i]    # abs vort at 950 mb
    rel = f.variables['rhprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,1] = rel[7,j,i]    # RH at 750 mb
    sfc = f.variables['pressfc']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,2] = sfc[j,i]      # surface pressure
    tmp = f.variables['tmpprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,3] = tmp[15,j,i]   # temperature at 400 mb
    tsf = f.variables['tmpsfc']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,4] = tsf[j,i]      # surface temperature
    ugr = f.variables['ugrdprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,5] = ugr[3,j,i]    # u-wind at 900 mb
            a2[i,j,6] = ugr[17,j,i]   # u-wind at 300 mb
    vgr = f.variables['vgrdprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,7] = vgr[3,j,i]    # v-wind at 900 mb
            a2[i,j,8] = vgr[17,j,i]   # v-wind at 300 mb
    hgt = f.variables['hgtprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,9] = hgt[3,j,i]    # geopotential at 850 mb
    wgr = f.variables['vvelprs']
    for i in range(a2.shape[0]):
        for j in range(a2.shape[1]):
            a2[i,j,10] = wgr[3,j,i]   # w-wind at 900 mb
            a2[i,j,11] = wgr[17,j,i]  # w-wind at 300 mb
    new_array = cv2.resize(a2, (IMG_SIZE, IMG_SIZE))
    #
    # normalize the data
    #
    #print('Number of channels to normalize is: ',number_channels)
    for var in range(number_channels):
        maxvalue = new_array[:,:,var].flat[np.abs(new_array[:,:,var]).argmax()]
        #print('Normalization factor for channel',var,', is: ',abs(maxvalue))
        new_array[:,:,var] = new_array[:,:,var]/abs(maxvalue)
    out_array = np.reshape(new_array, (-1, IMG_SIZE, IMG_SIZE, number_channels))
    #print('reshape new_array returns: ',out_array.shape)
    #input('Enter to continue...')
    return out_array

In [ ]:
#
# build an F1-score function for later use
#
def F1_score(y_true,y_prediction,true_class,true_threshold):
    T = len(y_true)
    if len(y_prediction) != T:
        print("Prediction and true label arrays have different size. Stop")
        return
    P = 0
    TP = 0 
    FN = 0
    TN = 0
    FP = 0
    for i in range(T):
        if y_true[i] == true_class:
            P = P + 1       
            if y_prediction[i] >= true_threshold:
                TP += 1 
            else:
                FN += 1
        else:
            if y_prediction[i] >= true_threshold:
                FP += 1 
            else:
                TN += 1            
    N = T - P    
    F1 = 2.*TP/(2.*TP + FP + FN)
    Recall = TP/float(TP+FN)
    if TP == 0 and FP == 0: 
        Precision = 0.
    else:    
        Precision = TP/float(TP+FP)
    return F1, Recall, Precision

In [6]:
#
# loop thru all best-saved CNN trained models and make a prediction. Note that prediction is applied one by one instead 
# of a batch input. 
#
DATADIR = "/N/slate/ckieu/deep-learning/data/ncep_binary_30x30_00h/testing"
bestmodels = ["3-conv-32-layer-0-dense.model_00h","3-conv-32-layer-1-dense.model_00h","3-conv-32-layer-2-dense.model_00h",
              "5-conv-32-layer-0-dense.model_00h","5-conv-32-layer-1-dense.model_00h","5-conv-32-layer-2-dense.model_00h"]
CATEGORIES = ["neg", "pos"]
F1_performance = []
for bestmodel in bestmodels:
    model = tf.keras.models.load_model(bestmodel)
    prediction_total = 0
    prediction_yes = 0
    prediction_history = []
    truth_history = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR,category)
        for img in tqdm(os.listdir(path)):    
            try:
                img_dir = DATADIR + '/' + category + '/' + img
                print('Processing image:', img_dir)
                #print('Input image dimension is: ',prepare(img_dir).shape)
                prediction = model.predict([prepare(img_dir)])
                print("TC formation prediction is",prediction,round(prediction[0][0]),CATEGORIES[round(prediction[0][0])])
                prediction_history.append(prediction[0][0])
                if round(prediction[0][0]) == 1:
                    prediction_yes += 1
                if category == "pos":
                    truth_history.append(1)
                else:
                    truth_history.append(0)
                prediction_total += 1    
                if prediction_total > 1000:
                    break
            except Exception as e:
                pass   
    #print(f"Quick check of prediction accuracy for yes event only is: {prediction_yes/prediction_total: .4f}") 
    #
    # Compute F1 score for each best model now
    #
    print(prediction_history)
    F1_performance.append([bestmodel,F1_score(truth_history,prediction_history,1,0.5)]) 

/N/project/pfec_climo/qmnguyen/tc_prediction/binary_datasets/ncep_WP_binary_0h/pos


  0%|                                                                                                                                                                 | 0/352 [00:00<?, ?it/s]

Processing image: /N/project/pfec_climo/qmnguyen/tc_prediction/binary_datasets/ncep_WP_binary_0h//pos/20140905_06_00_18.6_128.5_2014248N19129.nc
Input image dimension is:  (1, 30, 30, 12)
1/1 [==============================] - 0s 57ms/step


  0%|▍                                                                                                                                                        | 1/352 [00:05<32:20,  5.53s/it]

TC formation prediction is [[0.9999962]] 1 Yes
Processing image: /N/project/pfec_climo/qmnguyen/tc_prediction/binary_datasets/ncep_WP_binary_0h//pos/20100913_06_00_16.6_137.4_2010256N17137.nc
Input image dimension is:  (1, 30, 30, 12)
1/1 [==============================] - 0s 16ms/step


...

TC formation prediction is [[0.9999995]] 1 Yes
Processing image: /N/project/pfec_climo/qmnguyen/tc_prediction/binary_datasets/ncep_WP_binary_0h//pos/20151001_06_00_16.9_166.9_2015274N17167.nc
Input image dimension is:  (1, 30, 30, 12)
1/1 [==============================] - 0s 17ms/step


 14%|█████████████████████▌                                                                                                                                  | 50/352 [04:41<28:35,  5.68s/it]

TC formation prediction is [[0.999895]] 1 Yes
Processing image: /N/project/pfec_climo/qmnguyen/tc_prediction/binary_datasets/ncep_WP_binary_0h//pos/20160924_18_00_14.9_164.6_2016269N15165.nc
Input image dimension is:  (1, 30, 30, 12)
1/1 [==============================] - 0s 17ms/step


 14%|█████████████████████▌                                                                                                                                  | 50/352 [04:47<28:57,  5.75s/it]

TC formation prediction is [[0.7519042]] 1 Yes
Prediction accuracy for yes event is:  0.9020


In [7]:
#
# Print out the F1 performance of all models
#
print("F1, Recall, Precision for all models are:")
for i in range(len(bestmodels)):
    print("Model:", F1_performance[i])

[[0.9999962, 1], [0.6882587, 1], [0.9955371, 1], [0.85128695, 1], [0.97033226, 1], [0.09757158, 0], [0.99786943, 1], [1.0, 1], [0.99999934, 1], [0.9987997, 1], [0.96775466, 1], [0.55922717, 1], [0.9999991, 1], [0.9999858, 1], [1.0, 1], [0.99997646, 1], [0.9990219, 1], [0.99999243, 1], [0.99574035, 1], [0.99985635, 1], [0.99976224, 1], [0.008809361, 0], [0.7542709, 1], [0.98565555, 1], [0.9934654, 1], [0.00093312794, 0], [1.0, 1], [0.9999476, 1], [0.9929507, 1], [0.0015317168, 0], [0.9999999, 1], [0.6828572, 1], [0.9999997, 1], [0.9999999, 1], [0.993845, 1], [0.9964261, 1], [0.9963727, 1], [0.0017297732, 0], [0.9999232, 1], [0.99999267, 1], [0.99957865, 1], [0.98262095, 1], [0.99999493, 1], [0.9999934, 1], [0.99553144, 1], [0.991998, 1], [0.9999994, 1], [0.9999998, 1], [0.9999995, 1], [0.999895, 1], [0.7519042, 1]]
